In [3]:
import cv2
import numpy as np
 
# Load pre-trained YOLO model and its configuration
net = cv2.dnn.readNet(r"C:\Users\Dell\Downloads\archive\yolov3_2000.weights",r"C:\Users\Dell\Downloads\archive\yolov3.cfg")  # YOLOv3 model weights and config
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
 
# Load COCO names for object detection (COCO dataset class labels)
with open(r"C:\Users\Dell\Downloads\coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
 
# Initialize the webcam (0 for default camera)
cap = cv2.VideoCapture(0)
 
while True:
    ret, frame = cap.read()
    height, width, channels = frame.shape
 
    # Prepare the image for YOLO model input
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
 
    # Process each detected object
    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                # Get bounding box coordinates
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
 
    # Apply non-maxima suppression to remove overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
 
    # Draw boxes and labels
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
 
    # Display the frame with detected objects
    cv2.imshow("Real-Time Object Detection", frame)
 
    # Break the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
# Release camera and close window
cap.release()
cv2.destroyAllWindows()